In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
file = 'final.csv'
df = pd.read_csv(file)
df = pd.DataFrame(df)
df

,Год,Направление,Уровень безработицы,Уровень инфляции,Вступительный балл,Пол,Предыдущее образование (оценка),Перемещенный,ВВП,Дневная/вечерняя форма обучения,Возраст код,Количество абитуриентов
0,2022.0,0.000000,10.8,1.4,127.0,1,122.0,1,1.74,1,1,215
1,2023.0,1.000000,13.9,-0.3,142.0,1,160.0,1,0.79,1,0,252
2,2024.0,2.000000,10.8,1.4,125.0,1,122.0,1,1.74,1,0,225
3,2022.0,3.000000,9.4,-0.8,120.0,0,122.0,1,-3.12,1,1,331
4,2022.0,6.000000,15.5,2.8,128.0,0,142.0,1,-4.06,1,0,762
...,...,...,...,...,...,...,...,...,...,...,...,...
5452,2024.0,5.000000,7.6,2.6,117.0,0,133.0,1,0.32,0,4,574
5453,2024.0,4.000000,9.4,-0.8,100.0,0,133.0,0,-3.12,0,4,393
5454,2022.0,5.000000,12.7,3.7,160.0,0,160.0,0,-1.70,0,2,574
5455,2023.0,4.000000,10.8,1.4,123.0,0,133.0,0,1.74,0,4,393


In [13]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 

X = df[['Пол', 'Возраст код', 'Дневная/вечерняя форма обучения', 'Перемещенный', 'Вступительный балл', 'Предыдущее образование (оценка)']]
y = df['Количество абитуриентов']

In [14]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
# Параметры кросс-валидации
n_iter = 10  # Количество итераций
test_size = 0.2    # Размер тестовой выборки (20%)
mse_scores = []  # Список для хранения MSE на каждой итерации 
r2_scores = []   # Список для хранения R² на каждой итерации 

for i in range(n_iter): 
    # Случайное разбиение данных на обучающую и тестовую выборки 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=i) 

    # Создание и обучение модели (Линейная регрессия) 
    linear_model = LinearRegression() 
    linear_model.fit(X_train, y_train) 

    # Предсказание 
    y_pred = linear_model.predict(X_test) 

    # Оценка производительности модели 
    mse = mean_squared_error(y_test, y_pred)  # Среднеквадратичная ошибка
    r2 = r2_score(y_test, y_pred)              # Коэффициент детерминации

    mse_scores.append(mse) 
    r2_scores.append(r2)

    print(f'Итерация {i + 1}: MSE = {mse:.2f}, R² = {r2:.2f}') 

# Средние значения за все итерации 
mean_mse = np.mean(mse_scores) 
mean_r2 = np.mean(r2_scores)

print(f'Средняя MSE за {n_iter} итераций: {mean_mse:.2f}')
print(f'Средний R² за {n_iter} итераций: {mean_r2:.2f}')

Итерация 1: MSE = 49486.07, R² = 0.03
Итерация 2: MSE = 50519.30, R² = 0.03
Итерация 3: MSE = 46257.43, R² = 0.03
Итерация 4: MSE = 49902.93, R² = 0.03
Итерация 5: MSE = 48209.72, R² = 0.03
Итерация 6: MSE = 50191.20, R² = 0.04
Итерация 7: MSE = 45779.43, R² = 0.00
Итерация 8: MSE = 48340.91, R² = 0.01
Итерация 9: MSE = 47639.11, R² = 0.02
Итерация 10: MSE = 46164.43, R² = 0.01
Средняя MSE за 10 итераций: 48249.05
Средний R² за 10 итераций: 0.02


In [15]:
from sklearn.ensemble import RandomForestClassifier

# Параметры кросс-валидации
n_iter = 10  # Количество итераций
test_size = 0.2    # Размер тестовой выборки (20%)

accuracies_for = []  # Список для хранения точности на каждой итерации

for i in range(n_iter):
    # Случайное разбиение данных на обучающую и тестовую выборки
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=i)

    # Создание и обучение модели (Random Forest)
    forest_model = RandomForestClassifier(random_state=42)
    forest_model.fit(X_train, y_train)
        
    # Предсказание
    y_pred_for = forest_model.predict(X_test)
        
    # Оценка точности модели
    accuracy = accuracy_score(y_test, y_pred_for)
    accuracies_for.append(accuracy)
    print(f'Итерация {i + 1}: Точность модели = {accuracy:.2f}')

# Средняя точность за все итерации
mean_accuracy = np.mean(accuracies_for)
print(f'Средняя точность за {n_iter} итераций: {mean_accuracy:.2f}')

Итерация 1: Точность модели = 0.28
Итерация 2: Точность модели = 0.31
Итерация 3: Точность модели = 0.29
Итерация 4: Точность модели = 0.29
Итерация 5: Точность модели = 0.33
Итерация 6: Точность модели = 0.34
Итерация 7: Точность модели = 0.27
Итерация 8: Точность модели = 0.32
Итерация 9: Точность модели = 0.30
Итерация 10: Точность модели = 0.31
Средняя точность за 10 итераций: 0.30


In [16]:
from sklearn.tree import DecisionTreeClassifier

# Параметры кросс-валидации
n_iter = 10  # Количество итераций
test_size = 0.2    # Размер тестовой выборки (20%)

accuracies_tree = []  # Список для хранения точности на каждой итерации

for i in range(n_iter):
    # Случайное разбиение данных на обучающую и тестовую выборки
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=i)

    # Создание и обучение модели
    decision_tree_model = DecisionTreeClassifier(random_state=42)
    decision_tree_model.fit(X_train, y_train)

    # Предсказание 
    y_pred_tree = decision_tree_model.predict(X_test)

    # Оценка точности модели
    accuracy = accuracy_score(y_test, y_pred_tree)
    accuracies_tree.append(accuracy)
    print(f'Итерация {i + 1}: Точность модели = {accuracy:.2f}')

# Средняя точность за все итерации
mean_accuracy = np.mean(accuracies_tree)
print(f'Средняя точность за {n_iter} итераций: {mean_accuracy:.2f}')

Итерация 1: Точность модели = 0.27
Итерация 2: Точность модели = 0.29
Итерация 3: Точность модели = 0.27
Итерация 4: Точность модели = 0.26
Итерация 5: Точность модели = 0.30
Итерация 6: Точность модели = 0.30
Итерация 7: Точность модели = 0.25
Итерация 8: Точность модели = 0.29
Итерация 9: Точность модели = 0.28
Итерация 10: Точность модели = 0.29
Средняя точность за 10 итераций: 0.28


In [17]:
from xgboost import XGBRegressor

# Параметры кросс-валидации 
n_iter = 10  # Количество итераций 
test_size = 0.2  # Размер тестовой выборки (20%) 

mse_scores = []  # Список для хранения MSE на каждой итерации 
r2_scores = []   # Список для хранения R² на каждой итерации 

for i in range(n_iter): 
    # Случайное разбиение данных на обучающую и тестовую выборки 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=i) 

    # Создание и обучение модели XGBoost
    xgb_model = XGBRegressor(objective='reg:squarederror', random_state=42)
    xgb_model.fit(X_train, y_train) 

    # Предсказание 
    y_pred = xgb_model.predict(X_test) 

    # Оценка производительности модели 
    mse = mean_squared_error(y_test, y_pred)  # Среднеквадратичная ошибка
    r2 = r2_score(y_test, y_pred)              # Коэффициент детерминации

    mse_scores.append(mse) 
    r2_scores.append(r2)

    print(f'Итерация {i + 1}: MSE = {mse:.2f}, R² = {r2:.2f}') 

# Средние значения за все итерации 
mean_mse = np.mean(mse_scores) 
mean_r2 = np.mean(r2_scores)

print(f'Средняя MSE за {n_iter} итераций: {mean_mse:.2f}')
print(f'Средний R² за {n_iter} итераций: {mean_r2:.2f}')

Итерация 1: MSE = 48547.23, R² = 0.05
Итерация 2: MSE = 50274.27, R² = 0.03
Итерация 3: MSE = 46750.76, R² = 0.02
Итерация 4: MSE = 50697.96, R² = 0.01
Итерация 5: MSE = 47958.29, R² = 0.03
Итерация 6: MSE = 47350.46, R² = 0.10
Итерация 7: MSE = 47493.23, R² = -0.03
Итерация 8: MSE = 48258.62, R² = 0.01
Итерация 9: MSE = 48460.15, R² = 0.01
Итерация 10: MSE = 45062.76, R² = 0.03
Средняя MSE за 10 итераций: 48085.37
Средний R² за 10 итераций: 0.03


Задача прогнозирования - частный случай задачи регрессии.
Попробовать обучить с помощью:
1. многослойного перспетрона
2. радиально-базисной сетью
3. обобщенно-регрессионной сетью
4. нейросеть регрессии

In [ ]:
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "c:\Users\1645295\Documents\GitHub\diplom\myenv\lib\site-packages\tensorflow_core\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "c:\Users\1645295\Documents\GitHub\diplom\myenv\lib\site-packages\tensorflow_core\python\pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "c:\Users\1645295\Documents\GitHub\diplom\myenv\lib\site-packages\tensorflow_core\python\pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "C:\Users\1645295\AppData\Local\Programs\Python\Python37\lib\imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "C:\Users\1645295\AppData\Local\Programs\Python\Python37\lib\imp.py", line 342, in load_dynamic
    return _load(spec)
ImportError: DLL load failed: Произошел 

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "c:\Users\1645295\Documents\GitHub\diplom\myenv\lib\site-packages\tensorflow_core\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "c:\Users\1645295\Documents\GitHub\diplom\myenv\lib\site-packages\tensorflow_core\python\pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "c:\Users\1645295\Documents\GitHub\diplom\myenv\lib\site-packages\tensorflow_core\python\pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "C:\Users\1645295\AppData\Local\Programs\Python\Python37\lib\imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "C:\Users\1645295\AppData\Local\Programs\Python\Python37\lib\imp.py", line 342, in load_dynamic
    return _load(spec)
ImportError: DLL load failed: Произошел 

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "c:\Users\1645295\Documents\GitHub\diplom\myenv\lib\site-packages\tensorflow_core\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "c:\Users\1645295\Documents\GitHub\diplom\myenv\lib\site-packages\tensorflow_core\python\pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "c:\Users\1645295\Documents\GitHub\diplom\myenv\lib\site-packages\tensorflow_core\python\pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "C:\Users\1645295\AppData\Local\Programs\Python\Python37\lib\imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "C:\Users\1645295\AppData\Local\Programs\Python\Python37\lib\imp.py", line 342, in load_dynamic
    return _load(spec)
ImportError: DLL load failed: Произошел 